In [1]:
import SimpleITK as sitk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from ipywidgets import interact, IntSlider
import os

ID = '1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524522225658609808059'
num_subset = '0'

img_path = f'data/subset{num_subset}/subset{num_subset}/{ID}.mhd'
csv_path = 'data/annotations.csv'

itk_image = sitk.ReadImage(img_path)
ct_scan = sitk.GetArrayFromImage(itk_image) # (z, y, x)
origin = np.array(itk_image.GetOrigin())    # (x, y, z)
spacing = np.array(itk_image.GetSpacing())  # (x, y, z)

df = pd.read_csv(csv_path)
series_uid = os.path.basename(img_path).replace('.mhd', '')
nodules = df[df['seriesuid'] == series_uid]


In [2]:
def view_image(slice_idx):
    fig, ax = plt.subplots(figsize=(8, 8))
    
    ax.imshow(ct_scan[slice_idx], cmap='gray')
    ax.set_title(f"Slice {slice_idx} | UID: {series_uid}")
    ax.axis('off')
    
    for index, row in nodules.iterrows():
        nodule_center_world = np.array([row['coordX'], row['coordY'], row['coordZ']])
        nodule_diameter_mm = row['diameter_mm']
        nodule_radius_mm = nodule_diameter_mm / 2.0
        
        # Convert Center to Voxel Grid (x, y, z)
        nodule_center_vox = np.absolute(nodule_center_world - origin) / spacing
        
        # Calculate distance from this slice to the nodule center (in mm)
        slice_z_mm = origin[2] + slice_idx * spacing[2] 
        
        z_dist_vox = abs(slice_idx - nodule_center_vox[2])
        z_dist_mm = z_dist_vox * spacing[2]
        
        # CHECK: Does this slice intersect the nodule sphere?
        if z_dist_mm < nodule_radius_mm:
            # Radius of the circle at this specific slice intersection
            intersect_radius_mm = np.sqrt(nodule_radius_mm**2 - z_dist_mm**2)
            
            # Convert radius from mm to pixels (using X spacing, assuming XY isotropic)
            intersect_radius_pix = intersect_radius_mm / spacing[0]
            
            # Create the circle patch
            circle = Circle((nodule_center_vox[0], nodule_center_vox[1]), 
                            intersect_radius_pix, 
                            color='r', fill=False, linewidth=2)
            ax.add_patch(circle)
            
            ax.text(nodule_center_vox[0], nodule_center_vox[1] - intersect_radius_pix - 5, 
                    f"Nodule {row.name}", color='red', ha='center', fontsize=10)
    plt.show()


In [3]:
print(f"Found {len(nodules)} nodules in this scan.")
interact(view_image, slice_idx=IntSlider(min=0, max=ct_scan.shape[0]-1, step=1));

Found 2 nodules in this scan.


interactive(children=(IntSlider(value=0, description='slice_idx', max=160), Output()), _dom_classes=('widget-i…